# Visible Markov Models

Markov Models, as you now should know, are great models for determining a part-of-speech (POS) tag for a given string in a sentence. The best known Markov Model in the world for determining English POS tags is actually a Maximum Entropy Markov Model, which is essentially a Perceptron predictor, and it predicts with a 97-percent accuracy. However, visible Markov Models (MM) are much more lightweight and can achieve close to the same accuracy. The flow of a MM is similar to a Naive Bayes flow: (1) split a sentence into tokens, (2) take the token and multiply it by known percentages (such as odds that a token is a tag, or odds that a tag follows a tag), (3) predict that the token is whatever tag has the highest likelihood. Shall we dive in deeper?

# 1. Convert data to usable format

In machine learning, often the first step of any project is to take a raw data file and turn it into something usable. In this case, we will turn it into two dictionaries: one to track the odds of a token being a certain tag, and the other to track the odds of a tag following a certain tag. Let's take a look!

In [34]:
# store the probabilities in dictionaries... let's start with 2 to keep this simple
token_as_tag_probabilities = {} # odds a token exists as a given tag
tag_to_tag_probabilities = {} # odds a pos tag follows a given pos tag

# helper func to update our dictionaries
def add_to_dict(dictionary, key1, key2):
    if key1 in dictionary:
        dictionary[key1]["total"] += 1
        if key2 in dictionary[key1]:
            dictionary[key1][key2] += 1
        else:
            dictionary[key1][key2] = 1
    else:
        dictionary[key1] = {key2: 1, "total": 1}

# let's just track how many words and sentences are in the training data to get an idea of what "big" data is
word_count = 0
sentence_count = 1 # start at 1 to make sure first sentence is accounted for
with open("train.tagged") as training_data:
    for line in training_data:
        prev_pos = "" # represents first word in sentence (i.e. it has no previous pos tag)
        if "\t" in line:
            word_count += 1 # new word!
            word_and_pos = line.split("\t")
            word = word_and_pos[0].strip()
            pos = word_and_pos[1].strip()
            add_to_dict(token_as_tag_probabilities, word, pos)
            add_to_dict(tag_to_tag_probabilities, prev_pos, pos)
        else:
            sentence_count += 1 # start of a new sentence!
            prev_pos = "" # new sentence so reset the prev_pos

print("Awesome! Now let's see how much data we're working with...")
print("%d words from %d sentences!? WOAH!" % (word_count, sentence_count))
print("")
print("Let's check the odds of a random word being a certain tag...")
print("Odds for 'run':")
print(token_as_tag_probabilities["run"])

Awesome! Now let's see how much data we're working with...
56600 words from 2416 sentences!? WOAH!

Let's check the odds of a random word being a certain tag...
Odds for 'run':
{'VBP': 4, 'total': 12, 'VB': 5, 'VBN': 2, 'NN': 1}


# 2. Cleaning the data

This is a great start and all, but "run" seems like a fairly common word and still only appears 12 times in our giant data set. This could cause trouble for words that appear maybe once or twice only. Additionally, what happens if in testing we run into a word that we've never seen in the training set? Let's account for the unknown by considering words that appear twice or less in our training set as an unknown word.

In [35]:
unknown_tokens = []
for token in token_as_tag_probabilities:
    if token_as_tag_probabilities[token]["total"] <= 2:
        unknown_tokens.append(token)
unknown_token_as_tag_probabilities = {} # create a new dictionary to track the odds of an unknown token being a certain tag
for token in unknown_tokens:
    for tag in token_as_tag_probabilities[token]:
        if tag in unknown_token_as_tag_probabilities:
            unknown_token_as_tag_probabilities[tag] += token_as_tag_probabilities[token][tag]
        else:
            unknown_token_as_tag_probabilities[tag] = token_as_tag_probabilities[token][tag]
    token_as_tag_probabilities.pop(token, None) # remove the unknown token from the known token set
print(unknown_token_as_tag_probabilities)

{'CD': 502, 'total': 7019, 'NN': 1386, 'NNS': 782, 'VBN': 276, 'JJ': 1033, 'VB': 349, 'VBD': 229, 'RB': 237, 'NNP': 1514, 'VBG': 355, 'JJS': 12, 'IN': 28, 'VBZ': 148, 'VBP': 59, 'UH': 4, 'RBR': 3, 'NNPS': 22, 'RBS': 1, 'JJR': 25, 'DT': 6, 'WRB': 7, 'WDT': 2, 'PRP': 3, 'PRP$': 5, 'WP': 1, 'FW': 19, 'SYM': 1, 'RP': 2, 'TO': 1, ':': 2, 'MD': 4, '.': 1}


# 3. Now let's test!

The best part about (visible) Markov Models is how simple they are. Simply, just add up odds from a training set, then multiply those odds together in testing to form a prediction. Let's try it!

In [52]:
# be sure to track our accuracy!
correct_predictions = 0
total_predictions = 0

# helper func to make a prediction based on the previous pos tag and the given word
def make_prediction(prev_pos, word):
    probabilities = {}
    # is this a known word or unknown word?
    if word in token_as_tag_probabilities:
        for tag in token_as_tag_probabilities[word]:
            if tag != "total": # skip total (since it's not a POS tag)
                probabilities[tag] = float(token_as_tag_probabilities[word][tag]) / float(token_as_tag_probabilities[word]["total"])
    else:
        for tag in unknown_token_as_tag_probabilities:
            if tag != "total": # skip total (since it's not a POS tag)
                probabilities[tag] = float(unknown_token_as_tag_probabilities[tag]) / float(unknown_token_as_tag_probabilities["total"])
    for tag in tag_to_tag_probabilities[prev_pos]:
        if tag != "total": # skip total (since it's not a POS tag)
            if tag in probabilities:
                probabilities[tag] *= (float(tag_to_tag_probabilities[prev_pos][tag]) / float(tag_to_tag_probabilities[prev_pos]["total"]))
    highest_score = 0
    best_prediction = ""
    for tag in probabilities:
        if probabilities[tag] > highest_score:
            highest_score = probabilities[tag]
            best_prediction = tag
    return best_prediction

# this looks similar to before, huh?
with open("test.tagged") as training_data:
    for line in training_data:
        prev_pos = "" # represents first word in sentence (i.e. it has no previous pos tag)
        if "\t" in line:
            word_and_pos = line.split("\t")
            word = word_and_pos[0].strip()
            pos = word_and_pos[1].strip()
            predicted_pos = make_prediction(prev_pos, word)
            total_predictions += 1
            if predicted_pos == pos:
                correct_predictions += 1
        else:
            prev_pos = "" # new sentence so reset the prev_pos

print("Correctly predicted %d POS tags out of %d" % (correct_predictions, total_predictions))
percent_correct = (float(correct_predictions) / float(total_predictions)) * 100
print("That's %.2f%%!" % percent_correct)
print()
print("Not bad for a first go...")

Correctly predicted 48394 POS tags out of 56600
That's 85.50%!

Not bad for a first go...


# 4. Now how can we make this even better?

Let's discuss.